# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier

## Read the CSV and Perform Basic Data Cleaning

In [10]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)
dummify = df.select_dtypes(exclude = "number").drop(columns = "loan_status")
df = pd.get_dummies(df, columns = list(dummify.columns), drop_first = True)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_w,next_pymnt_d_May-2019,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,0,0,1,1,0,0,1,1,1,0
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,0,0,0,1,0,1,1,1,0
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,0,0,0,1,0,1,1,1,0
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,0,0,1,0,1,0,1,1,1,0
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,1,0,0,0,0,0,1,1,1,0


## Split the Data into Training and Testing

In [11]:
# Create our features
X = df.drop(columns = "loan_status")

# Create our target
y = df[["loan_status"]]

In [12]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_w,next_pymnt_d_May-2019,application_type_Joint App
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.526309,0.106747,0.357659,0.373992,0.148001,0.451066,0.177238,0.876121,0.616839,0.139660
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.499311,0.308793,0.479314,0.483865,0.355104,0.497603,0.381873,0.329446,0.486161,0.346637
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
# Check the balance of our target values
y["loan_status"].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [14]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [15]:
# Create the StandardScaler instance
data_scaler = StandardScaler()

In [16]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
data_scaler.fit(X_train)

StandardScaler()

In [17]:
# Scale the training and testing data
X_train = data_scaler.transform(X_train)
X_test = data_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [18]:
# Resample the training data with the BalancedRandomForestClassifier
brfc = BalancedRandomForestClassifier(n_estimators = 100, random_state = 1)
brfc = brfc.fit(X_train, y_train)
y_prediction = brfc.predict(X_test)

In [19]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_prediction)

0.7838202356607264

In [20]:
# Display the confusion matrix
confusion_matrix(y_test, y_prediction)

array([[   61,    29],
       [ 1885, 15230]], dtype=int64)

In [21]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_prediction))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.68      0.89      0.06      0.78      0.59        90
   low_risk       1.00      0.89      0.68      0.94      0.78      0.62     17115

avg / total       0.99      0.89      0.68      0.94      0.78      0.62     17205



In [31]:
# List the features sorted in descending order by feature importance
important_features = brfc.feature_importances_
sorted(zip(brfc.feature_importances_, X.columns), reverse = True)#[:30]

[(0.08580624143234544, 'total_rec_prncp'),
 (0.06563576420896067, 'total_pymnt'),
 (0.06416732535322989, 'total_rec_int'),
 (0.054464138522986, 'last_pymnt_amnt'),
 (0.05000266541854427, 'total_pymnt_inv'),
 (0.03321765753537096, 'int_rate'),
 (0.028196541250718698, 'issue_d_Jan-2019'),
 (0.01765803168227423, 'dti'),
 (0.01735210474043815, 'mo_sin_old_rev_tl_op'),
 (0.017082867391693596, 'il_util'),
 (0.0168476686588051, 'annual_inc'),
 (0.01660723586097285, 'max_bal_bc'),
 (0.016106613012209475, 'out_prncp_inv'),
 (0.016023260763994414, 'installment'),
 (0.015833671052610254, 'total_bc_limit'),
 (0.015801610254658267, 'revol_bal'),
 (0.01569015385487817, 'tot_hi_cred_lim'),
 (0.015288386704382149, 'mo_sin_old_il_acct'),
 (0.014928927667020572, 'bc_open_to_buy'),
 (0.014733303228284787, 'bc_util'),
 (0.014196293898555964, 'out_prncp'),
 (0.0135906114254383, 'tot_cur_bal'),
 (0.013568442787161867, 'avg_cur_bal'),
 (0.013318183128891608, 'total_rev_hi_lim'),
 (0.012856191653745384, 'mths

### Easy Ensemble Classifier

In [32]:
# Train the Classifier
ensemble_classifier = EasyEnsembleClassifier(n_estimators = 100, random_state = 1)
ensemble_classifier = ensemble_classifier.fit(X_train, y_train)
y_prediction = ensemble_classifier.predict(X_test)

In [33]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_prediction)

0.9050199629954231

In [34]:
# Display the confusion matrix
confusion_matrix(y_test, y_prediction)

array([[   77,    13],
       [  779, 16336]], dtype=int64)

In [35]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_prediction))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.86      0.95      0.16      0.90      0.81        90
   low_risk       1.00      0.95      0.86      0.98      0.90      0.82     17115

avg / total       0.99      0.95      0.86      0.97      0.90      0.82     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The Easy Ensemble Classifier yielded the highest balanced accuracy score.
    - 0.9050199629954231

2. Which model had the best recall score?

    The Easy Ensemble Classifier yielded the the highest recall score 
    - high risk = 0.86
    - low risk  = 0.95
    - avg/total = 0.95

3. Which model had the best geometric mean score?

    The Easy Ensemble Classifier yielded the highest recall score
    - high risk = 0.90
    - low risk  = 0.90
    - avg/total = 0.90

4. What are the top three features?

    The top 3 features are:
    - 'total_rec_prncp'
    - 'total_pymnt'
    - 'total_rec_int'